In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import sklearn.cluster as cluster
from scipy.spatial.distance import cdist, pdist
import matplotlib.pyplot as plt
% matplotlib inline

ImportError: dlopen(/Users/adriandahlin/anaconda/lib/python2.7/site-packages/fiona/ogrext.so, 2): Library not loaded: @rpath/libnetcdf.11.dylib
  Referenced from: /Users/adriandahlin/anaconda/lib/libgdal.20.dylib
  Reason: Incompatible library version: libgdal.20.dylib requires version 12.0.0 or later, but libnetcdf.11.dylib provides version 11.0.0

In [ ]:
obesity = pd.read_csv('IHME_USA_OBESITY_PHYSICAL_ACTIVITY_2001_2011.csv')

In [ ]:
obesity.head(10)

In [ ]:
obesity.columns

In [ ]:
obesity.shape

In [ ]:
obesity.drop([u'fips', u'merged_fips', u'County', u'Prevalence 2001 lower bound (%)', 
              u'Prevalence 2001 upper bound (%)', u'Prevalence 2002 lower bound (%)', 
              u'Prevalence 2002 upper bound (%)', u'Prevalence 2003 lower bound (%)',
              u'Prevalence 2003 upper bound (%)', u'Prevalence 2004 lower bound (%)', 
              u'Prevalence 2004 upper bound (%)', u'Prevalence 2005 lower bound (%)',
              u'Prevalence 2005 upper bound (%)', u'Prevalence 2006 lower bound (%)', 
              u'Prevalence 2006 upper bound (%)', u'Prevalence 2007 lower bound (%)',
              u'Prevalence 2007 upper bound (%)', u'Prevalence 2008 lower bound (%)', 
              u'Prevalence 2008 upper bound (%)', u'Prevalence 2009 lower bound (%)',
              u'Prevalence 2009 upper bound (%)', u'Prevalence 2010 lower bound (%)', 
              u'Prevalence 2010 upper bound (%)', u'Prevalence 2011 lower bound (%)',
              u'Prevalence 2011 upper bound (%)', u'Difference 2001 to 2009 lower bound',
              u'Difference 2001 to 2009 upper bound'], axis = 1, inplace = True)

In [ ]:
obesity.head()

In [ ]:
obesity.Outcome.unique()

In [ ]:
obesity.dtypes

In [ ]:
male_obesity = obesity[(obesity.Sex == 'Male') & (obesity.Outcome == 'Obesity')]
female_obesity = obesity[(obesity.Sex == 'Female') & (obesity.Outcome == 'Obesity')]

In [ ]:
male_obesity.head()

In [ ]:
f_obesity_state.head()

In [ ]:
m_obesity_state = male_obesity.groupby(male_obesity.State).mean()
f_obesity_state = female_obesity.groupby(female_obesity.State).mean()

In [ ]:
f_clust_01 = pd.DataFrame(f_obesity_state['Prevalence 2001 (%)'])

In [ ]:
f_clust_01.reset_index(inplace = True)

In [ ]:
f_clust_01.head()

In [ ]:
def elbow(data, K):
    km1 = [KMeans(n_clusters = k).fit(data) for k in K]
    centroids = [k.cluster_centers_ for k in km1]
    distK = [cdist(data, cent, 'euclidean') for cent in centroids]
    globals()['dist'] = [np.min(D, axis = 1) for D in distK]
    avSS = [sum(d) / data.shape[0] for d in dist]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, avSS, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within-cluster sum of squares')
    plt.title('Elbow for KMeans clustering')

In [ ]:
elbow(f_clust_01.iloc[:, 1:], range(1, 10))

In [ ]:
n_clust = 4
KM = cluster.KMeans(n_clusters = n_clust)
KM.fit(f_clust_01.iloc[:, 1:])
KM_clust = KM.predict(f_clust_01.iloc[:, 1:])

In [ ]:
KM_clust

In [ ]:
res_KM = pd.concat((f_clust_01, pd.DataFrame(KM_clust, columns = ['Cluster_01'])), axis = 1)
res_KM.head()

In [ ]:
f_clust_11 = pd.DataFrame(f_obesity_state['Prevalence 2011 (%)'])
f_clust_11.reset_index(inplace = True)

KM = cluster.KMeans(n_clusters = n_clust)
KM.fit(f_clust_11.iloc[:, 1:])
KM_clust = KM.predict(f_clust_11.iloc[:, 1:])
res_KM_f11 = pd.concat((f_clust_11, pd.DataFrame(KM_clust, columns = ['Cluster_11'])), axis = 1)
res_KM_f11.head()

In [ ]:
f_clust = pd.merge(res_KM, res_KM_f11, on = 'State')

In [ ]:
f_clust.head()

In [ ]:
f_clust[f_clust['Cluster_01'] == f_clust['Cluster_11']]

In [ ]:
# Males
m_clust_01 = pd.DataFrame(m_obesity_state['Prevalence 2001 (%)'])
m_clust_01.reset_index(inplace = True)

KM = cluster.KMeans(n_clusters = n_clust, random_state = 123)
KM.fit(m_clust_01.iloc[:, 1:])
KM_clust = KM.predict(m_clust_01.iloc[:, 1:])
res_KM_m01 = pd.concat((m_clust_01, pd.DataFrame(KM_clust, columns = ['Cluster_01'])), axis = 1)
res_KM_m01.head()

In [ ]:
m_clust_11 = pd.DataFrame(m_obesity_state['Prevalence 2011 (%)'])
m_clust_11.reset_index(inplace = True)

KM = cluster.KMeans(n_clusters = n_clust, random_state = 123)
KM.fit(m_clust_11.iloc[:, 1:])
KM_clust = KM.predict(m_clust_11.iloc[:, 1:])
res_KM_m11 = pd.concat((m_clust_11, pd.DataFrame(KM_clust, columns = ['Cluster_11'])), axis = 1)
res_KM_m11.head()

In [ ]:
m_clust = pd.merge(res_KM_m01, res_KM_m11, on = 'State')

In [ ]:
m_clust.to_csv('USObesity.csv')

In [ ]:
m_clust[m_clust['Cluster_01'] != m_clust['Cluster_11']]

In [ ]:
res_KM_m01.Cluster_01.value_counts()

In [ ]:
res_KM_m11.Cluster_11.value_counts()

In [ ]:
# Visualize 
import shapefile as shp
from shapely.geometry.polygon import Polygon
from descartes import PolygonPatch

zipsf = shp.Reader('states.shp')
zipsf_records=zipsf.records()
zipsf_shape=zipsf.shapeRecords()

In [ ]:
zip_shapes = gpd.GeoDataFrame.from_file("states.shp")

In [ ]:
zip_shapes.head()

In [ ]:
zip_shapes.columns

In [ ]:
zip_shapes.drop([u'DRAWSEQ', u'STATE_ABBR', u'STATE_FIPS', u'SUB_REGION'], axis=1, inplace=True)

In [ ]:
zip_shapes.head()

In [ ]:
zip_shapes.rename(columns = {u'STATE_NAME': 'State'}, inplace=True)

In [ ]:
m_clust = pd.merge(m_clust, zip_shapes, on = 'State')

In [ ]:
m_clust.to_csv('USObesity.csv')

In [ ]:
m_clust = gpd.GeoDataFrame(m_clust)

In [ ]:
type(m_clust)

In [ ]:
f, ax = plt.subplots(figsize=(9, 9))
m_clust.plot(column='Cluster_01', categorical=True, 
        k=10, cmap='hot_r', linewidth=0.1, ax=ax, 
        edgecolor='black', legend=True)
ax.set_axis_off()
plt.title("Male obesity 2001")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(9, 9))
m_clust.plot(column='Cluster_11', categorical=True, 
        k=10, cmap='hot_r', linewidth=0.1, ax=ax, 
        edgecolor='black', legend=True)
ax.set_axis_off()
plt.title("Male obesity 2011")
plt.show()

In [ ]:
f_clust = pd.merge(f_clust, zip_shapes, on = 'State')

In [ ]:
f_clust = gpd.GeoDataFrame(f_clust)

In [ ]:
f_clust.head()

In [ ]:
f, ax = plt.subplots(figsize=(9, 9))
f_clust.plot(column='Cluster_01', categorical=True, 
        k=10, cmap='hot_r', linewidth=0.1, ax=ax, 
        edgecolor='black', legend=True)
ax.set_axis_off()
plt.title("Female obesity 2001")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(9, 9))
f_clust.plot(column='Cluster_11', categorical=True, 
        k=10, cmap='hot_r', linewidth=0.1, ax=ax, 
        edgecolor='black', legend=True)
ax.set_axis_off()
plt.title("Female obesity 2011")
plt.show()